In [3]:
#Preamble
import ipywidgets as widgets
import io
import os
import pandas as pd
import requests
from matplotlib.pyplot import *
import rpy2
import rpy2.robjects as robjects
import numpy
import textwrap
import requests
import json
import us

pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)




### Introduction
This notebook will help you answer the question of "Does a certain kind of police activity happen disproportionately (as compared to the town's population) to different racial groups?"

### **Step #0:** Lets get some census data to compare to

If you have a CSV with the racial makeup of your town, upload that. Otherwise, we (eventually) will automatically pull census data.

Usually this can easily be found by going to http://data.census.gov and typing your town's name in the search bar.  Then, select the 'RACE' data file, and click 'CSV' to download a CSV.

In [15]:
robjects.r('''
        # create a function `fetchCensusRaceData`
        fetchCensusRaceData <- function(state,county) {
            c(state,county)
        }
        ''')
r_FetchCensusRaceData = robjects.r['fetchCensusRaceData']

In [ ]:
%matplotlib inline


selectCensusDataMethod = widgets.RadioButtons(
    options=['I have a CSV', 'Fetch census data for me'],
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Do you already have demographic data?',
)

#for later use
censusRaceDropDown = widgets.Dropdown()

def load_data0():
    global townRace
    #townRaceRaw = pd.read_csv(os.path.join("data/",fileSelect0.value),index_col=[0])
    townRaceRaw = pd.read_csv(io.BytesIO(fileUpload0.value[-1].content),index_col=[0])
    townRaceRaw.columns=['Population']
    townRaceRaw['Population']=townRaceRaw['Population'].apply(lambda s: s.replace(',',''))
    townRaceRaw['Population']=townRaceRaw['Population'].astype('int')
    totalTownPopulation = townRaceRaw.filter(like='Total',axis='index').iloc[0,0]
    townRace = pd.DataFrame([["White",townRaceRaw.filter(like='White',axis='index')['Population'].sum()],
                          ["Black or African American", townRaceRaw.filter(like='Black',axis='index')['Population'].sum()],
                          ["American Indian or Alaskan Native", townRaceRaw.filter(like='American Indian',axis='index')['Population'].sum()],
                          ["Asian", townRaceRaw.filter(like='Asian',axis='index')['Population'].sum()],
                          ["Native Hawaiian or Other Pacific Islander",townRaceRaw.filter(like='Native Hawaiian',axis='index')['Population'].sum()],
                          ["Some Other Race", townRaceRaw.filter(like='Other',axis='index')['Population'].sum()]])
    townRace.columns=['Race','Population']
    townRace['Percentage']=townRace['Population']/totalTownPopulation

    with outputWidget0:
        display("Successfully loaded the datafile " + fileUpload0.value[-1].name)

    displayCensusData()



def displayCensusData():
    censusRaceDropDown.options=townRace['Race']
    with outputWidget4:
        display(censusRaceDropDown)
    with outputWidget1:
        #display("The total population of your town is " + str(totalTownPopulation))
        townRace.plot.bar(x='Race',y='Percentage')
        show()

outputWidget0 = widgets.Output(layout={'border': '1px solid black'})
outputWidget1 = widgets.Output(layout={'border': '1px solid black'})
fileUpload0 = widgets.FileUpload()
fileUpload0.observe(lambda b: load_data0(), names='value')
apiText = widgets.Text() #description='Census API Key')

def selection():
    outputWidget0.clear_output()
    with outputWidget0:
        #display(widgets.Label("You have chosen "+selectCensusDataMethod.value))
        if selectCensusDataMethod.value == 'I have a CSV':
            display(fileUpload0)
        if selectCensusDataMethod.value == 'Fetch census data for me':
            #display(r_FetchCensusRaceData("AL","Mobile"))
            display(widgets.HBox([widgets.Label("Enter your census API Key:"), apiText]))
            display(fetchButton)
            display(statusBox)
            display(widgets.HBox([widgets.Label("Select the state your town is in:"),stateDropDown]))
            display(widgets.HBox([widgets.Label("Select your town: "),placeDropDown]))
            
stateDropDown = widgets.Dropdown()
placeDropDown = widgets.Dropdown()
statusBox = widgets.VBox()
def fetchClick(api_key):
    statusBox.children += (widgets.Label("Fetching with census key " + api_key + " ... (this may take a moment)"),)
    global censusData
    censusData = fetchCensusData(api_key)
    statusBox.children+= (widgets.Label("Complete!"),)
    if not censusData.empty:
        stateDropDown.options = censusData.groupby('State Code').groups.keys()



def stateSelected():
    #placeDropDown.options = sorted(censusData[censusData['State Code']==stateDropDown.value]['NAME'].values.tolist())
    placeDropDown.options = sorted(censusData[censusData['State Code']==stateDropDown.value].index.values.tolist())
stateDropDown.observe(lambda b: stateSelected(), names='value')

def placeSelected():
    global townRace
    with outputWidget1:
        display(str(censusData.loc[placeDropDown.value]['White']))
        display(str(censusData.loc[placeDropDown.value]['Black']))
        display(str(censusData.loc[placeDropDown.value]['AIAN']))
        display(str(censusData.loc[placeDropDown.value]['Asian']))
        display(str(censusData.loc[placeDropDown.value]['NHPI']))
        display(str(censusData.loc[placeDropDown.value]['Some Other Race']))
    townRace = pd.DataFrame([["White",censusData.loc[placeDropDown.value]['White']],
                          ["Black or African American", censusData.loc[placeDropDown.value]['Black']],
                          ["American Indian or Alaskan Native", censusData.loc[placeDropDown.value]['AIAN']],
                          ["Asian", censusData.loc[placeDropDown.value]['Asian']],
                          ["Native Hawaiian or Other Pacific Islander", censusData.loc[placeDropDown.value]['NHPI']],
                          ["Some Other Race", censusData.loc[placeDropDown.value]['Some Other Race']] ])
    townRace.columns=['Race','Population']
    townRace['Population']=townRace['Population'].astype('int')
    townRace['Percentage']=townRace['Population']/int(censusData.loc[placeDropDown.value]['Total Population'])
    #with outputWidget1:
    #    display(townRace)
    #    display("The total population of " + placeDropDown.value + " is " + str(censusData.loc[placeDropDown.value]['Total Population']))
    #    townRace.plot.bar(x='Race',y='Percentage')
    #    show()
    displayCensusData()
placeDropDown.observe(lambda b: placeSelected(), names='value')

fetchButton = widgets.Button(description="Fetch Census Data")
fetchButton.on_click(lambda b: fetchClick(apiText.value))

selectCensusDataMethod.observe(lambda b: selection(), names='value')
#Make sure our event handler runs the first time 
selection()

display(selectCensusDataMethod)
display(outputWidget0)
display(outputWidget1)

In [ ]:
#Dictionary of Census columns
censusDict = {
'B01003_001E': 'Total Population',
'B01001_002E': 'Male',
'B01001_026E': 'Female',
'B02008_001E': 'White',
'B02009_001E': 'Black',
'B02010_001E': 'AIAN',
'B02011_001E': 'Asian',
'B02012_001E': 'NHPI',
'B02013_001E': 'Some Other Race',
'B03001_002E': 'Nonhispanic',
'B03001_003E': 'Hispanic',
'B06011_001E': 'Median Income',
'B06012_002E': 'Below poverty level',
'B06012_003E': 'Below 1.5 poverty level',
'B23025_003E': 'Labor force',
'B23025_004E': 'Employed',
'B23025_005E': 'Unemployed',
'B01001_003E': 'Male 0-4',
'B01001_004E': 'Male 5-9',
'B01001_005E': 'Male 10-14',
'B01001_006E': 'Male 15-17',
'B01001_007E': 'Male 18-19',
'B01001_008E': 'Male 20',
'B01001_009E': 'Male 21',
'B01001_010E': 'Male 22-24',
'B01001_011E': 'Male 25-29',
'B01001_012E': 'Male 30-34',
'B01001_013E': 'Male 35-39',
'B01001_014E': 'Male 40-44',
'B01001_015E': 'Male 45-49',
'B01001_016E': 'Male 50-54',
'B01001_017E': 'Male 55-59'
}

In [ ]:
#Returns a data frame of census
def fetchCensusData(api_key):
    year='2020'
    dsource='acs' # the survey i.e. decennial census, acs, etc
    dseries='acs5' # a dataset within the survey
    #cols="""NAME, B02008_001E, B01003_001E, B01001_002E, B01001_026E, B02008_001E, B02009_001E, B02010_001E, B02011_001E, B02012_001E, 
    #        B02013_001E, B03001_002E, B03001_003E, B06011_001E, B06012_002E, B06012_003E, B23025_003E, B23025_004E, B23025_005E, 
    #        B01001_003E, B01001_004E, B01001_005E, B01001_006E, B01001_007E, B01001_008E, B01001_009E, B01001_010E, B01001_011E, 
    #        B01001_012E, B01001_013E, B01001_014E, B01001_015E, B01001_016E, B01001_017E """ # census variables
    #cols=cols.replace(' ','')
    #cols=cols.replace('\n','')
    #cols=cols.replace('\t','')
    cols="NAME,"+','.join(censusDict.keys())
    #for debug
    #cols="NAME,B02008_001E"
    state='*' # ansi fips codes for states; use asterisk * for all states
    place='*' # ansi fips codes cities / towns; use asterisk * for all places

    #api_key="60de298763b23e26f4c1b27e16a524ff026bfc7c"


    base_url = f'https://api.census.gov/data/{year}/{dsource}/{dseries}'

    # for sub-geography within larger geography - geographies must nest
    data_url = f'{base_url}?get={cols}&for=place:{place}&in=state:{state}&key={api_key}'

    raw = requests.get(data_url).text
    #Make sure the request went through
    if raw.find('Invalid Key',0,100) != -1:
        display("Error: invalid API key")
        return pd.DataFrame()

    #else unpack the json
    data = json.loads(raw)

    #The first entry has the column headers
    columns = data.pop(0)
    #Load into a dataframe
    df=pd.DataFrame(data, columns=columns)
    #Rename using our dictionary
    df.rename(columns=censusDict, inplace=True)
    #Look up the postal code abbreviations for the states
    df['State Code']= df.apply(lambda row: us.states.lookup(str(row['state'])).abbr, axis=1)
    df=df.set_index('NAME')
    return df


### **Step #1:** Load the police data

Upload a CSV file containing data on police encounters here. We assume each row is a unique encounter with police.

In [4]:
outputWidget = widgets.Output(layout={'border': '1px solid black'})

fileUpload1 = widgets.FileUpload()
display(fileUpload1)

outputWidget3 = widgets.Output(layout={'border': '1px solid black'})
raceDropDown=widgets.Dropdown()
def load_data():
    global arrests
    arrests = pd.read_csv(io.BytesIO(fileUpload1.value[-1].content),index_col=[0])
    global raceDropDown
    raceDropDown.options=arrests.columns
    with outputWidget3:
        display(raceDropDown)
    
    with outputWidget:
        display("Successfully loaded the datafile " + fileUpload1.value[-1].name )

fileUpload1.observe(lambda b: load_data(), names='value')
display(outputWidget)

Dropdown(options=('Durham_arrests_charges.csv', 'Durham_census_race_table.csv'), value='Durham_arrests_charges…

Button(description='Load data file', style=ButtonStyle())

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

<IPython.core.display.Javascript object>

What kind of encounter does this datafile represent (e.g. arrest, traffic stop, etc.)?

In [ ]:
encounterType = widgets.Text( description='Encounter type:', style= {'description_width': 'initial'})
display(encounterType)

If you would like to preview your data, select how many rows you would like to preview from the dropdown.

In [5]:
outputWidget2 = widgets.Output(layout={'border': '1px solid black'})
# Take a first look:
numRows = widgets.Dropdown(
    options=['5', '10', '15', '20'],
    description='Number:',
    disabled=False,
)
display(numRows)
def preview_data():
    outputWidget2.clear_output()
    with outputWidget2:
        display(arrests.sample(int(numRows.value)))
numRows.observe(lambda b: preview_data(), names='value')
display(outputWidget2)

Dropdown(description='Number:', options=('5', '10', '15', '20'), value='5')

Button(description='Preview', style=ButtonStyle())

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

### **Step #2:** Which column has the racial data?

Which of these columns represents the race of the person described in the police report? 

In [ ]:
%matplotlib inline
display(outputWidget3)

#For later use
policeRaceSelectMultiple = widgets.SelectMultiple()
def raceDropdownChange():
    policeRaceSelectMultiple.options = arrests.groupby(raceDropDown.value).groups.keys()
    with outputWidget4:
        display(policeRaceSelectMultiple)
    with outputWidget3:
        arrests[raceDropDown.value].value_counts(normalize=True).plot.bar()
        show()
raceDropDown.observe(lambda b: raceDropdownChange(),names='value')

In order to compare, we need to somehow match up Census racial categories with police racial categories.
Select a census racial category from the dropdown, and then select which categories from the police data should match to it.
When finished, click the button to confirm.

In [ ]:
#May refactor later. Here is a first go
outputWidget4 = widgets.Output(layout={'border': '1px solid black'})

#Make a dropdown for race categories from census
raceMapDict={}
def policeSelectChange():
    global raceMapDict
    with outputWidget4:
        if censusRaceDropDown.value:
            raceMapDict[censusRaceDropDown.value]=policeRaceSelectMultiple.value
            
            if policeRaceSelectMultiple.value:
                display("Police categories " + " and ".join(policeRaceSelectMultiple.value) + " are now mapped to " + censusRaceDropDown.value)
            else:
                display("No police categories are mapped to "+censusRaceDropDown.value)
        else:
            display("Select a census category first")

policeRaceSelectMultiple.observe(lambda b: policeSelectChange(), names='value')

display(outputWidget4)

In [ ]:
outputWidget5 = widgets.Output(layout={'border': '1px solid black'})

dictButton = widgets.Button(
    description='Click to confirm mapping of categories',
)

display(dictButton)

def displayDict():
    outputWidget5.clear_output()
    with outputWidget5:
        #Check if they accidentally mapped a category multiple places
        for x in set([ item for l in raceMapDict.values() for item in l]):
            if [ item for l in raceMapDict.values() for item in l].count(x)> 1:
                display("Warning: " + str(x) + " is mapped to multiple categories")
        #Display the mapping for verification
        for key in raceMapDict.keys():
            display(key +" is mapped to " + " and ".join(raceMapDict[key]))

        X = raceMapDict.keys()
        Ytown=[]
        Ypolice=[]
        townRace.set_index('Race', inplace = True)
        for key in X:
            Ytown.append(townRace.loc[key,'Percentage'])
            Ypolice.append(sum([arrests[raceDropDown.value].value_counts(normalize=True)[v] for v in raceMapDict[key]]))

        X_axis = numpy.arange(len(X))

        if encounterType.value:
            encounterString = encounterType.value
        else:
            encounterString = "Police encounters"
        
        matplotlib.pyplot.bar(X_axis - 0.2, Ytown, 0.4, label = 'Town')
        matplotlib.pyplot.bar(X_axis + 0.2, Ypolice, 0.4, label = encounterString)
        
        #Handle long category names gracefully
        f = lambda x: textwrap.fill(x, 10)
        matplotlib.pyplot.xticks(X_axis, map(f,X))
        matplotlib.pyplot.xlabel("Race")
        matplotlib.pyplot.ylabel("Percentage")
        matplotlib.pyplot.title("Comparison of town demographics to " + encounterString)
        matplotlib.pyplot.legend()
        matplotlib.pyplot.show()
dictButton.on_click(lambda b: displayDict())


display(outputWidget5)